In [ ]:
%load_ext autoreload
%autoreload 2

import pickle
import math

# The deformation module library is not automatically installed yet, we need to add its path manually
import sys
sys.path.append("../../")

import numpy as np
import matplotlib.pyplot as plt
import torch

import implicitmodules.torch as dm

torch.set_default_tensor_type(torch.FloatTensor)

In [ ]:
# In this notebook, we try to fit a model of growth constant.
# The dataset consists of bended rectangles, done by applying an implicit module of order 1 with known linear growth law.
# To generate the dataset, simply execute "python3 generate_bendings.py bendings.pickle nb_of_elem_in_dataset".
# We then fit a polynomial model of growth constants of order 2.

In [ ]:
# Load our data
data = []
with open("bendings.pickle", 'rb') as f:
    data = pickle.load(f)

template = data[0][0]
template = torch.unique_consecutive(template, dim=0)


true_implicit1_points = data[0][1]
true_C = data[0][2]
bendings = data[1:]

# Make sure each points in each data element are consecutively unique (needed when dealing varifold attachment atm).
dataset = [torch.unique_consecutive(target, dim=0) for target in list(zip(*bendings))[0]]

targets = dataset[:15]

print("Dataset size: {size}".format(size=len(targets)))

In [ ]:
%matplotlib qt5

# Display our data set and template
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
for target in targets:
    plt.plot(target[:, 0].numpy(), target[:, 1].numpy(), color='grey', lw=0.5)
plt.plot(template[:, 0].numpy(), template[:, 1].numpy(), '--', color='black', lw=2.)
plt.plot(true_implicit1_points[:, 0].numpy(), true_implicit1_points[:, 1].numpy(), 'x')
plt.axis('equal')
plt.show()

In [ ]:
# Defines the implicit module of order 1 that will be used by the atlas.
sigma_implicit1 = 1.5

implicit1_points = true_implicit1_points
implicit1_R = torch.eye(2).repeat(implicit1_points.shape[0], 1, 1)
C_init = torch.ones(implicit1_points.shape[0], 2, 1)
implicit1 = dm.DeformationModules.ImplicitModule1(2, true_implicit1_points.shape[0], sigma_implicit1, C_init, nu=0.1, gd=(implicit1_points.clone().requires_grad_(), implicit1_R.clone().requires_grad_()))

# Initial coefficient for the polynomial model of the deformation constants
abc_init = torch.zeros(6, 2)
abc_init[0] = torch.ones(2)

In [ ]:
# The polynomial model we will try to fit on our deformation constants
def pol_order_2(pos, a, b, c, d, e, f):
    return a + b*pos[:, 0] + c*pos[:, 1] + d*pos[:, 0]**2 + e*pos[:, 1]**2 + f*pos[:, 0]*pos[:, 1]

# Callback called by the atlas. Serves as glue for our model of deformation constants.
def callback_compute_c(init_manifold, modules, parameters):
    abc = parameters[-1]
    a = abc[0].unsqueeze(1)
    b = abc[1].unsqueeze(1)
    c = abc[2].unsqueeze(1)
    d = abc[3].unsqueeze(1)
    e = abc[4].unsqueeze(1)
    f = abc[5].unsqueeze(1)
    modules[1]._ImplicitModule1Base__C = pol_order_2(implicit1_points, a, b, c, d, e, f).transpose(0, 1).unsqueeze(2)

In [ ]:
# Creation of the atlas. Uses L2 norm attachment for now.
atlas = dm.Models.Atlas(template, [implicit1], [dm.Attachment.L2NormAttachment()], len(targets), lam=10000., other_parameters=[abc_init.clone().requires_grad_()], model_precompute_callback=callback_compute_c)

In [ ]:
# Fit the atlas
fitter = dm.Models.ModelFittingScipy(atlas, 1.)
costs = fitter.fit(targets, 150, options={'shoot_method': 'rk4', 'shoot_it': 10})

In [ ]:
plt.plot(range(len(costs)), costs)
plt.show()

In [ ]:
# Retrieve the optimised coeffcient and then compare the generated growth constraint to the true one.
abc_fit = atlas.parameters[-1].detach()
print(abc_fit)
C_fit = pol_order_2(implicit1_points, abc_fit[0].unsqueeze(1), abc_fit[1].unsqueeze(1), abc_fit[2].unsqueeze(1), abc_fit[3].unsqueeze(1), abc_fit[4].unsqueeze(1), abc_fit[5].unsqueeze(1)).t().unsqueeze(2)

# The closer to 1. it is, the better.
print(torch.dot((true_C/torch.norm(true_C)).flatten(), (C_fit/torch.norm(C_fit)).flatten()))